In [33]:
import pandas as pd

file_path = 'C:\\Users\\User\\OneDrive\\Desktop\\python_developer\\partizan_2022\\data\\partizan_2022_cleaned.csv'

df = pd.read_csv(file_path)

In [34]:
# filter 
players = df[df['player'] != 'PARTIZAN MOZZART BET BELGRADE'].copy()
team = df[df['player'] == 'PARTIZAN MOZZART BET BELGRADE'].copy()

In [35]:
# points per possession (PPP)
# Scoring efficiency per possession used, highlighting players who maximize points from their offensive opportunities.
# formula: PPP = Points / (FGA + 0.44 * FTA + TO)
players['field_goals_attempted'] = players['two_points_attempted'] + players['three_points_attempted']
players['possessions_used'] = players['field_goals_attempted'] + 0.44 * players['free_throws_attempted'] + players['turnovers']
players['points_per_possession'] = players['points'] / players['possessions_used']


# handle division by zero (players with no possessions used)
players['points_per_possession'] = players['points_per_possession'].fillna(0).replace([float('inf'), -float('inf')], 0)


# average PPP per player
avg_PPP = players.groupby('player')['points_per_possession'].mean().sort_values(ascending=False)


print("Top 5 Players by Points per Possession (PPP):")
print(avg_PPP.head(5))

Top 5 Players by Points per Possession (PPP):
player
Lessort Mathias    1.232044
Exum Dante         1.146356
Punter Kevin       1.126225
Smailagic Alen     1.123254
Leday Zach         1.062734
Name: points_per_possession, dtype: float64


In [52]:
# Verify Smailagic 
smailagic = players[players['player'] == 'Smailagic Alen']
print(smailagic[['points', 'field_goals_attempted', 'free_throws_attempted', 'turnovers', 'points_per_possession']].mean())

points                   6.222222
field_goals_attempted    3.822222
free_throws_attempted    1.244444
turnovers                0.555556
points_per_possession    1.193479
dtype: float64


In [37]:
# calculate Usage Rate (USG%)
# Usage Rate (USG%): Percentage of team possessions used by a player, adjusted for playing time, calculated as:
# formula: USG% = 100 * ((FGA + 0.44 * FTA + TO) / (FGA_team + 0.44 * FTA_team + TO_team)) * (TS / 2400)


# calculate total shot attempts because of merging below
players['field_goals_attempted'] = players['two_points_attempted'] + players['three_points_attempted']
team['field_goals_attempted'] = team['two_points_attempted'] + team['three_points_attempted']


# merge team stats to players
players = players.merge(team[['game', 'field_goals_attempted', 'free_throws_attempted', 'turnovers', 'total_seconds']], 
                        on='game', suffixes=('', '_team'))

# calculate Usage Rate
game_seconds = 2400  # 40 minutes * 60 seconds
players['usage_rate'] = 100 * ((players['field_goals_attempted'] + 0.44 * players['free_throws_attempted'] + players['turnovers']) /
                               (players['field_goals_attempted_team'] + 0.44 * players['free_throws_attempted_team'] + players['turnovers_team'])) * \
                              (players['total_seconds'] / game_seconds)
     
                                       
# handle division by zero (players with no team stats)                              
players['usage_rate'] = players['usage_rate'].fillna(0).replace([float('inf'), -float('inf')], 0)

# average USG% per player
avg_usg = players.groupby('player')['usage_rate'].mean().sort_values(ascending=False)


print("Top 5 Players by Usage Rate (USG%):")
print(avg_usg.head(5))

Top 5 Players by Usage Rate (USG%):
player
Punter Kevin       13.644535
Leday Zach         11.260251
Lessort Mathias     9.040352
Exum Dante          8.773679
Nunnally James      6.643273
Name: usage_rate, dtype: float64
